In [1]:
import pandas as pd
import warnings

warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

from hypex.dataset import Dataset, ExperimentData, InfoRole, TreatmentRole, TargetRole
from hypex.comparators import TTest, KSTest, GroupSizes, GroupDifference
from hypex.utils import SpaceEnum
from hypex.splitters import AASplitter
from hypex.analyzers import OneAASplitAnalyzer
from hypex.experiments import CycledExperiment, Experiment, OnRoleExperiment

ImportError: cannot import name 'OneAASplitAnalyzer' from 'hypex.analyzers' (c:\Projects\HypEx\hypex\analyzers\__init__.py)

In [2]:
data = Dataset(
    roles={
        "user_id": InfoRole(int),
        "treat": TreatmentRole(int),
        "pre_spends": TargetRole(),
        "post_spends": TargetRole()
    }, data="data.csv",
)
data

      user_id  signup_month  treat  pre_spends  post_spends   age gender  \
0           0             0      0       488.0   414.444444   NaN      M   
1           1             8      1       512.5   462.222222  26.0    NaN   
2           2             7      1       483.0   479.444444  25.0      M   
3           3             0      0       501.5   424.333333  39.0      M   
4           4             1      1       543.0   514.555556  18.0      F   
...       ...           ...    ...         ...          ...   ...    ...   
9995     9995            10      1       538.5   450.444444  42.0      M   
9996     9996             0      0       500.5   430.888889  26.0      F   
9997     9997             3      1       473.0   534.111111  22.0      F   
9998     9998             2      1       495.0   523.222222  67.0      F   
9999     9999             7      1       508.0   475.888889  38.0      F   

        industry  
0     E-commerce  
1     E-commerce  
2      Logistics  
3     E-com

In [3]:
aa = Experiment(
    executors=[
        AASplitter(),
        OnRoleExperiment(
            executors=[
                GroupSizes(grouping_role=TreatmentRole(), space=SpaceEnum.additional),
                GroupDifference(grouping_role=TreatmentRole(), space=SpaceEnum.additional),
                TTest(grouping_role=TreatmentRole(), space=SpaceEnum.additional),
                KSTest(grouping_role=TreatmentRole(), space=SpaceEnum.additional),
            ],
            role=TargetRole(),
        )
    ]
)
experiment = CycledExperiment(n_iterations=20, inner_executor=aa, analyzer=OneAASplitAnalyzer())
ed = ExperimentData(data)
res = experiment.execute(ed)

C:\Users\User\PycharmProjects\HypEx\hypex\dataset\backends\pandas_backend.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data[column_name] = self.data[column_name].astype(type_name)
C:\Users\User\PycharmProjects\HypEx\hypex\dataset\backends\pandas_backend.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data[column_name] = self.data[column_name].astype(type_name)
C:\Users\User\PycharmProjects\HypEx\hypex\dataset\backends\pandas_backend.py:81: SettingWithCopyWarning: 
A value is trying to

In [4]:
res.additional_fields

     AASplitter╰╰0 AASplitter╰╰1 AASplitter╰╰2 AASplitter╰╰3 AASplitter╰╰4  \
0                A             B             A             A             B   
1                A             A             A             B             A   
2                A             A             A             B             A   
3                A             B             A             A             B   
4                B             B             A             B             A   
...            ...           ...           ...           ...           ...   
9995             A             B             A             B             B   
9996             A             A             B             A             B   
9997             B             A             A             B             B   
9998             A             B             B             A             A   
9999             B             A             B             A             A   

     AASplitter╰╰5 AASplitter╰╰6 AASplitter╰╰7 AASplitter╰╰8 AA

In [ ]:
res.analysis_tables

In [5]:
res_analyzer = OneAASplitAnalyzer().execute(res)
res_analyzer.analysis_tables['OneAASplitAnalyzer╰╰']

   TTest p-value  TTest pass  KSTest p-value  KSTest pass  mean test score
0       0.399466        0.05        0.501559          0.0         0.467528